<a href="https://colab.research.google.com/github/Haebuk/AutoML/blob/main/MNIST%2Bwandb%2Bsweeps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow-gpu==2.0.0-rc1
import tensorflow as tf

     |████████████████████████████████| 380.5MB 30kB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 501kB 53.5MB/s 
     |████████████████████████████████| 4.3MB 52.2MB/s 


In [2]:
!pip install -q wandb
import wandb 
from wandb.keras import WandbCallback

     |████████████████████████████████| 2.1MB 7.2MB/s 
     |████████████████████████████████| 163kB 32.7MB/s 
     |████████████████████████████████| 102kB 12.1MB/s 
     |████████████████████████████████| 133kB 33.7MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 


In [10]:
default_config = {
    'learning_rate': 0.02,
    'dropout_rate': 0.3,
    'hidden1': 256,
    'activation1': 'tanh'
}
wandb.init(project='mnist-tf2-sweep', config=default_config)
config = wandb.config
#config.learning_rate = 0.02
#config.dropout_rate = 0.3
#config.hidden1 = 256
#config.activation1 = 'tanh'

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0,  x_test / 255.0

11493376/11490434 [==============================] - 0s 0us/step


In [11]:
%%wandb
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(config.hidden1, activation=config.activation1),
    tf.keras.layers.Dropout(config.dropout_rate),
    tf.keras.layers.Dense(10, activation='softmax')
])
opt = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)
model.compile(optimizer=opt,
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [6]:
model.fit(x_train, y_train, 
          validation_data = (x_test, y_test), # 중간중간에 성능 체크
          epochs=5, callbacks=[WandbCallback()])

model.evaluate(x_test, y_test, verbose=2)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 7s 115us/sample - loss: 0.8169 - accuracy: 0.8215 - val_loss: 0.4689 - val_accuracy: 0.9089
Epoch 2/5
60000/60000 [==============================] - 5s 90us/sample - loss: 0.8599 - accuracy: 0.8462 - val_loss: 0.5953 - val_accuracy: 0.9036
Epoch 3/5
60000/60000 [==============================] - 5s 84us/sample - loss: 0.8276 - accuracy: 0.8622 - val_loss: 0.4721 - val_accuracy: 0.9148
Epoch 4/5
60000/60000 [==============================] - 5s 87us/sample - loss: 0.8325 - accuracy: 0.8660 - val_loss: 0.5612 - val_accuracy: 0.9034
Epoch 5/5
60000/60000 [==============================] - 5s 86us/sample - loss: 0.8098 - accuracy: 0.8715 - val_loss: 0.4638 - val_accuracy: 0.9247
10000/1 - 1s - loss: 0.2494 - accuracy: 0.9247


[0.4638038050765637, 0.9247]

In [15]:
!wandb agent kade/mnist-tf2-sweep/a26havxq

wandb: Starting wandb agent 🕵️
2021-04-08 07:58:32,025 - wandb.wandb_agent - INFO - Running runs: []
2021-04-08 07:58:33,505 - wandb.wandb_agent - INFO - Agent received command: run
2021-04-08 07:58:33,505 - wandb.wandb_agent - INFO - Agent starting run with config:
	activation1: tanh
	dropout_rate: 0.476049826185822
	hidden1: 410
	learning_rate: 0.03898200262159715
2021-04-08 07:58:33,506 - wandb.wandb_agent - INFO - About to run command: /usr/bin/env python train.py --activation1=tanh --dropout_rate=0.476049826185822 --hidden1=410 --learning_rate=0.03898200262159715
wandb: Currently logged in as: kade (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: Tracking run with wandb version 0.10.25
wandb: Syncing run rose-sweep-10
wandb: ⭐️ View project at https://wandb.ai/kade/mnist-tf2-sweep
wandb: 🧹 View sweep at https://wandb.ai/kade/mnist-tf2-sweep/sweeps/a26havxq
wandb: 🚀 View run at https://wandb.ai/kade/mnist-tf2